In [1]:
import os
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as imbl_pipe
from imblearn.over_sampling import SMOTE

In [2]:
df = pd.read_csv(os.path.join("../Resources", "analytical_base_table.csv"))
print(f"Dataframe dimensions: {df.shape}")
df.head()

Dataframe dimensions: (10000, 11)


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
# target variable
y = df.Exited
X = df.drop(['Exited'], axis=1)
print(X.shape, y.shape)

(10000, 10) (10000,)


In [4]:
#Numerical features
num_columns = X.select_dtypes(include='number').columns.tolist()
num_columns

['CreditScore',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary']

In [5]:
#Categorical features
cat_columns = X.select_dtypes(include='object').columns.tolist()
cat_columns

['Geography', 'Gender']

In [6]:
def class_count(a):
    counter=Counter(a)
    kv=[list(counter.keys()),list(counter.values())]
    dff = pd.DataFrame(np.array(kv).T, columns=['Exited','Count'])
    dff['Count'] = dff['Count'].astype('int64')
    dff['%'] = round(dff['Count'] / a.shape[0] * 100, 2)
    return dff.sort_values('Count',ascending=False)

In [7]:
class_count(y)

,Exited,Count,%
1,0,7963,79.63
0,1,2037,20.37


In [8]:
random_state = 10
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                    random_state=random_state,
                                                   stratify=df.Exited)
#X_train, X_test, y_train, and y_test
print(len(X_train), len(X_test), len(y_train), len(y_test))

7000 3000 7000 3000


In [9]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7000 entries, 8061 to 4741
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      7000 non-null   int64  
 1   Geography        7000 non-null   object 
 2   Gender           7000 non-null   object 
 3   Age              7000 non-null   int64  
 4   Tenure           7000 non-null   int64  
 5   Balance          7000 non-null   float64
 6   NumOfProducts    7000 non-null   int64  
 7   HasCrCard        7000 non-null   int64  
 8   IsActiveMember   7000 non-null   int64  
 9   EstimatedSalary  7000 non-null   float64
dtypes: float64(2), int64(6), object(2)
memory usage: 601.6+ KB


In [10]:
num_features = [] 
for i in num_columns:
    location = X.columns.get_loc(i)
    num_features.append(location)
print(num_features)  

[0, 3, 4, 5, 6, 7, 8, 9]


In [11]:
cat_features = []
for i in cat_columns:
    location = X.columns.get_loc(i)
    cat_features.append(location)
print(cat_features)  

[1, 2]


In [12]:
preprocess = make_column_transformer(
    (MinMaxScaler(), num_features),
    (OneHotEncoder(sparse=False), cat_features)
)


In [13]:
from sklearn.svm import SVC 
model = imbl_pipe(preprocess,
                  SVC(random_state=random_state))

In [14]:
from sklearn.model_selection import GridSearchCV
param_grid = {'svc__kernel' : ['linear', 'rbf', 'poly', 'sigmoid'],
              'svc__C': [0.0005,0.001, 0.01, 0.1, 0.5],
              'svc__gamma': [5, 1, 0.1, 0.01]}
grid = GridSearchCV(model, param_grid, verbose=3, cv= 5, n_jobs=4)

In [15]:
X_train = X_train.values
X_test = X_test.values

In [16]:
grid.fit(X_train, y_train)

Fitting 5 folds for each of 80 candidates, totalling 400 fits


/Users/tp/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/tp/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/tp/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/tp/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and 

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('minmaxscaler',
                                                                         MinMaxScaler(),
                                                                         [0, 3,
                                                                          4, 5,
                                                                          6, 7,
                                                                          8,
                                                                          9]),
                                                                        ('onehotencoder',
                                                                         OneHotEncoder(sparse=False),
                                                                         [1,
                                                                          2])])),
                                       ('svc', SVC(random_state=10))]),
             n_jobs=4,
             param_grid={'svc__C': [0.0005, 0.001, 0.01, 0.1, 0.5],
                         'svc__gamma': [5, 1, 0.1, 0.01],
                         'svc__kernel': ['linear', 'rbf', 'poly', 'sigmoid']},
             verbose=3)

In [17]:
print(grid.best_params_)

{'svc__C': 0.5, 'svc__gamma': 5, 'svc__kernel': 'poly'}


In [18]:
print(grid.best_score_)

0.8552857142857142


In [19]:
print(f"Training Score: {grid.score(X_train, y_train)}")
print(f"Testing Score: {grid.score(X_test, y_test)}")

Training Score: 0.8748571428571429
Testing Score: 0.865


In [20]:
predictions = grid.predict(X_test)
predictions

array([0, 0, 0, ..., 0, 0, 0])

In [21]:
# metrics
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(y_test, predictions)
print(cm)

[[2312   77]
 [ 328  283]]


In [22]:
cm = np.around(cm / cm.sum(axis=1)[:, np.newaxis], 2)
print(cm)

[[0.97 0.03]
 [0.54 0.46]]


In [23]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.88      0.97      0.92      2389
           1       0.79      0.46      0.58       611

    accuracy                           0.86      3000
   macro avg       0.83      0.72      0.75      3000
weighted avg       0.86      0.86      0.85      3000



In [24]:
predictions

array([0, 0, 0, ..., 0, 0, 0])

In [25]:
pred = grid.predict(X_test[:1])

In [26]:
print(f"Predicted classes: {pred}")
print(f"Actual labels: {list(y_test[:1])}")

Predicted classes: [0]
Actual labels: [1]


In [27]:
from sklearn.svm import SVC 
model_s = imbl_pipe(preprocess,
                  SMOTE(sampling_strategy='auto', random_state=random_state),
                  SVC(random_state=random_state))

In [28]:
from sklearn.model_selection import GridSearchCV

param_grid = {'svc__kernel' : ['linear', 'rbf', 'poly', 'sigmoid'],
              'svc__C': [0.0005,0.001, 0.01, 0.1, 0.5],
              'svc__gamma': [5, 1, 0.1, 0.01]}
grid_s = GridSearchCV(model_s, param_grid, verbose=3, cv= 5, n_jobs=4)

In [29]:
grid_s.fit(X_train, y_train)

Fitting 5 folds for each of 80 candidates, totalling 400 fits


/Users/tp/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/tp/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/tp/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/tp/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and 

[CV 4/5] END svc__C=0.0005, svc__gamma=5, svc__kernel=linear;, score=0.796 total time=   0.2s
[CV 1/5] END svc__C=0.0005, svc__gamma=5, svc__kernel=rbf;, score=0.796 total time=   0.3s
[CV 5/5] END svc__C=0.0005, svc__gamma=5, svc__kernel=rbf;, score=0.796 total time=   0.3s
[CV 1/5] END svc__C=0.0005, svc__gamma=5, svc__kernel=sigmoid;, score=0.796 total time=   0.3s
[CV 5/5] END svc__C=0.0005, svc__gamma=5, svc__kernel=sigmoid;, score=0.796 total time=   0.3s
[CV 5/5] END svc__C=0.0005, svc__gamma=1, svc__kernel=linear;, score=0.796 total time=   0.2s
[CV 4/5] END svc__C=0.0005, svc__gamma=1, svc__kernel=rbf;, score=0.796 total time=   0.3s
[CV 3/5] END svc__C=0.0005, svc__gamma=1, svc__kernel=poly;, score=0.796 total time=   0.2s
[CV 2/5] END svc__C=0.0005, svc__gamma=1, svc__kernel=sigmoid;, score=0.796 total time=   0.4s
[CV 1/5] END svc__C=0.0005, svc__gamma=0.1, svc__kernel=linear;, score=0.796 total time=   0.2s
[CV 2/5] END svc__C=0.0005, svc__gamma=0.1, svc__kernel=linear;, s

/Users/tp/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/tp/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/tp/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/tp/anaconda3/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the exe

[CV 2/5] END svc__C=0.0005, svc__gamma=5, svc__kernel=linear;, score=0.796 total time=   0.2s
[CV 3/5] END svc__C=0.0005, svc__gamma=5, svc__kernel=rbf;, score=0.796 total time=   0.3s
[CV 2/5] END svc__C=0.0005, svc__gamma=5, svc__kernel=poly;, score=0.849 total time=   0.3s
[CV 5/5] END svc__C=0.0005, svc__gamma=5, svc__kernel=poly;, score=0.856 total time=   0.3s
[CV 3/5] END svc__C=0.0005, svc__gamma=5, svc__kernel=sigmoid;, score=0.796 total time=   0.3s
[CV 2/5] END svc__C=0.0005, svc__gamma=1, svc__kernel=linear;, score=0.796 total time=   0.2s
[CV 1/5] END svc__C=0.0005, svc__gamma=1, svc__kernel=rbf;, score=0.796 total time=   0.3s
[CV 5/5] END svc__C=0.0005, svc__gamma=1, svc__kernel=rbf;, score=0.796 total time=   0.3s
[CV 1/5] END svc__C=0.0005, svc__gamma=1, svc__kernel=sigmoid;, score=0.796 total time=   0.4s
[CV 5/5] END svc__C=0.0005, svc__gamma=1, svc__kernel=sigmoid;, score=0.796 total time=   0.4s
[CV 3/5] END svc__C=0.0005, svc__gamma=0.1, svc__kernel=rbf;, score=0.

...

[CV 4/5] END svc__C=0.5, svc__gamma=0.01, svc__kernel=linear;, score=0.714 total time=   0.9s
[CV 2/5] END svc__C=0.5, svc__gamma=0.01, svc__kernel=rbf;, score=0.692 total time=   1.5s
[CV 5/5] END svc__C=0.5, svc__gamma=0.01, svc__kernel=rbf;, score=0.698 total time=   1.5s
[CV 4/5] END svc__C=0.5, svc__gamma=0.1, svc__kernel=sigmoid;, score=0.604 total time=   1.9s
[CV 3/5] END svc__C=0.5, svc__gamma=0.01, svc__kernel=linear;, score=0.723 total time=   0.9s
[CV 1/5] END svc__C=0.5, svc__gamma=0.01, svc__kernel=rbf;, score=0.685 total time=   1.5s
[CV 4/5] END svc__C=0.5, svc__gamma=0.01, svc__kernel=rbf;, score=0.699 total time=   1.5s
[CV 2/5] END svc__C=0.5, svc__gamma=0.01, svc__kernel=poly;, score=0.477 total time=   1.3s


/Users/tp/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/tp/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


[CV 5/5] END svc__C=0.5, svc__gamma=0.01, svc__kernel=linear;, score=0.731 total time=   1.0s
[CV 3/5] END svc__C=0.5, svc__gamma=0.01, svc__kernel=rbf;, score=0.696 total time=   1.4s
[CV 1/5] END svc__C=0.5, svc__gamma=0.01, svc__kernel=poly;, score=0.553 total time=   1.3s
[CV 3/5] END svc__C=0.5, svc__gamma=0.01, svc__kernel=poly;, score=0.513 total time=   1.3s


/Users/tp/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/tp/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/tp/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/tp/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and 

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('minmaxscaler',
                                                                         MinMaxScaler(),
                                                                         [0, 3,
                                                                          4, 5,
                                                                          6, 7,
                                                                          8,
                                                                          9]),
                                                                        ('onehotencoder',
                                                                         OneHotEncoder(sparse=False),
                                                                         [1,
                                                                          2])])),
                                       ('smote', SMOTE(random_state=10)),
                                       ('svc', SVC(random_state=10))]),
             n_jobs=4,
             param_grid={'svc__C': [0.0005, 0.001, 0.01, 0.1, 0.5],
                         'svc__gamma': [5, 1, 0.1, 0.01],
                         'svc__kernel': ['linear', 'rbf', 'poly', 'sigmoid']},
             verbose=3)

In [30]:
print(grid_s.best_params_)

{'svc__C': 0.5, 'svc__gamma': 5, 'svc__kernel': 'poly'}


In [31]:
print(grid_s.best_score_)

0.7865714285714285


In [32]:
print(f"TrainingScore: {grid_s.score(X_train, y_train)}")
print(f"TestingScore: {grid_s.score(X_test, y_test)}")

TrainingScore: 0.811
TestingScore: 0.7753333333333333


In [33]:
predictions_s = grid_s.predict(X_test)
predictions_s

array([1, 0, 0, ..., 1, 0, 0])

In [34]:
# metrics
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(y_test, predictions_s)
print(cm)

[[1871  518]
 [ 156  455]]


In [35]:
cm = np.around(cm / cm.sum(axis=1)[:, np.newaxis], 2)
print(cm)

[[0.78 0.22]
 [0.26 0.74]]


In [36]:
print(classification_report(y_test, predictions_s))

              precision    recall  f1-score   support

           0       0.92      0.78      0.85      2389
           1       0.47      0.74      0.57       611

    accuracy                           0.78      3000
   macro avg       0.70      0.76      0.71      3000
weighted avg       0.83      0.78      0.79      3000



In [37]:
predictions_s

array([1, 0, 0, ..., 1, 0, 0])

In [38]:
pred_s = grid.predict(X_test[:1])

In [39]:
print(f"Predicted class: {pred_s}")
print(f"Actual Label: {list(y_test[:1])}")

Predicted class: [0]
Actual Label: [1]


In [40]:
import joblib
filename_nos = '../models/SVM_model_nos.sav'
joblib.dump(grid, filename_nos)
filename_s = '../models/SVM_model_s.sav'
joblib.dump(grid_s, filename_s)

['../models/SVM_model_s.sav']

In [41]:
svm_model_nos = joblib.load(filename_nos)
print(svm_model_nos.score(X_test, y_test))
svm_model_s = joblib.load(filename_s)
print(svm_model_s.score(X_test, y_test))

0.865
0.7753333333333333
